In [1]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="hyperopt", 
           entity="benchmark-nlp",
           name='fake news datasets rf') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os
#Move back to the root directory of the project
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42
OPT_ITER=10

## Load data

In [5]:
dl = DataLoader(['fake_news'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization

train_gossipcop, val_gossipcop, _ = data_splitter(data['gossipcop'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)


100%|█████████████████████████████████████████████████████████████████████████████| 5323/5323 [00:08<00:00, 593.35it/s]


18590 rows preprocessed in 268.0978093147278 seconds


In [6]:
train_coaid, val_coaid, _ = data_splitter(data['CoAID'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 val_split=0.2,
                                 test_split=0.25,
                                 seed=SEED)
# train_liar, val_liar, _ = data_splitter(data['liar'],
#                                  preprocessor, 
#                                  create_val_set=True,   #No validation set is provided
#                                  seed=SEED)

5457 rows preprocessed in 12.108634233474731 seconds


In [7]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 35.2 s
Wall time: 1min 24s


In [9]:
embedded_train_gossipcop = fasttext.generate_sentence_embeddings(train_gossipcop['text'])
embedded_val_gossipcop = fasttext.generate_sentence_embeddings(val_gossipcop['text'])
embedded_train_gossipcop['label'] = train_gossipcop['label'].to_list()
embedded_val_gossipcop['label'] = val_gossipcop['label'].to_list()

Starting to generate sentence embeddings


100%|████████████████████████████████████████████████████████████████████████████| 11897/11897 [05:54<00:00, 33.60it/s]


Starting to generate sentence embeddings


100%|██████████████████████████████████████████████████████████████████████████████| 2975/2975 [01:22<00:00, 36.12it/s]


In [8]:
embedded_train_coaid = fasttext.generate_sentence_embeddings(train_coaid['text'])
embedded_val_coaid = fasttext.generate_sentence_embeddings(val_coaid['text'])
embedded_train_coaid['label'] = train_coaid['label'].to_list()
embedded_val_coaid['label'] = val_coaid['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 3273/3273 [00:27<00:00, 117.06it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████████| 819/819 [00:06<00:00, 119.05it/s]


In [10]:
embedded_train_liar = fasttext.generate_sentence_embeddings(train_liar['text'])
embedded_val_liar = fasttext.generate_sentence_embeddings(val_liar['text'])
embedded_train_liar['label'] = train_liar['label'].to_list()
embedded_val_liar['label'] = val_liar['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 10269/10269 [00:23<00:00, 435.15it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 1284/1284 [00:02<00:00, 436.19it/s]


## Hyperopt

In [9]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/rf_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [10]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'n_estimators': {'min': 10,
   'max': 200,
   'distribution': 'int_uniform'},
  'max_features': {'values': ['log2', 'sqrt']},
  'random_state': {'value': 42}}}

####  politifact   Tf-Idf

#### gossipcop

In [14]:
#Don't forget to name the sweep instance  
name = 'rf_tfidf_gossipcop' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_tfidf(config = None,
          train=train_gossipcop, #Change here
          val=val_gossipcop): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state)
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 11:29:51] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: gpm3oikj
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/gpm3oikj


[codecarbon WARNING @ 11:29:55] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 37epbru4 with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 35
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.8111
AUC-PC,0.88986
accuracy,0.81277
f1 macro,0.71678


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 0y5d8t0j with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 63
wandb: 	random_state: 42


...

wandb: Agent Starting Run: nmvi79gg with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 113
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.78823
AUC-PC,0.87386
accuracy,0.7805
f1 macro,0.63302


wandb: Agent Starting Run: nszco1rl with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 153
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.7955
AUC-PC,0.87655
accuracy,0.7805
f1 macro,0.63406


0.004469255448202312

In [15]:
#Don't forget to name the sweep instance   
name = 'rf_ft_gossipcop' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_gossipcop, #CHANGE HERE
          val=embedded_val_gossipcop): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state)
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 11:59:43] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 8730l14f
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/8730l14f


[codecarbon WARNING @ 11:59:46] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: pmlwwr6c with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 17
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.75566
AUC-PC,0.85717
accuracy,0.78387
f1 macro,0.68089


wandb: Agent Starting Run: wsjf4g4a with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 188
wandb: 	random_state: 42


...

wandb: Agent Starting Run: tbc9styn with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 140
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.79023
AUC-PC,0.87943
accuracy,0.79966
f1 macro,0.69677


wandb: Agent Starting Run: td31rsle with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 94
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.7887
AUC-PC,0.87906
accuracy,0.79966
f1 macro,0.6982


0.0014062016857814795

####  fake and real news dataset



In [11]:
#Don't forget to name the sweep instance  
name = 'rf_tfidf_coaid' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_tfidf(config = None,
          train=train_coaid, #Change here
          val=val_coaid): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state)
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 12:06:44] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: n7viahvr
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/n7viahvr


[codecarbon WARNING @ 12:06:50] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: rwal7yu3 with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 197
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.97007
AUC-PC,0.99392
accuracy,0.93407
f1 macro,0.87255


wandb: Agent Starting Run: uau7pxw3 with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 186
wandb: 	random_state: 42


...

wandb: Agent Starting Run: ly0c34se with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 27
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.96826
AUC-PC,0.99276
accuracy,0.92796
f1 macro,0.8612


wandb: Agent Starting Run: b0h7tqs1 with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 161
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.97664
AUC-PC,0.9953
accuracy,0.93895
f1 macro,0.88351


0.0016385528600349343

In [12]:
#Don't forget to name the sweep instance   
name = 'rf_ft_coaid' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_coaid, #CHANGE HERE
          val=embedded_val_coaid): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state)
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 12:17:49] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: c3mog94m
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/c3mog94m


[codecarbon WARNING @ 12:17:52] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: uaqkj2g5 with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 51
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.95645
AUC-PC,0.99039
accuracy,0.9243
f1 macro,0.84761


wandb: Agent Starting Run: j7ysrgad with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 134
wandb: 	random_state: 42


...

wandb: Agent Starting Run: 5jobll0x with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 27
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.9599
AUC-PC,0.99072
accuracy,0.92796
f1 macro,0.8612


wandb: Agent Starting Run: c3vh3ph0 with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 11
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.93668
AUC-PC,0.98237
accuracy,0.90354
f1 macro,0.80913


0.0013534707882262326

#### liar

In [16]:
#Don't forget to name the sweep instance  
name = 'rf_tfidf_liar' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_tfidf(config = None,
          train=train_liar, #Change here
          val=val_liar): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        config = wandb.config
        vec = TfidfVectorizer()
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state)
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 12:09:12] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: ld27797m
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/ld27797m


[codecarbon WARNING @ 12:09:15] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: elq26b8a with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 41
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.24143
f1 macro,0.21857


wandb: Agent Starting Run: zlc1cgb1 with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 58
wandb: 	random_state: 42


...

wandb: Agent Starting Run: 2a41ike2 with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 41
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.24143
f1 macro,0.21857


wandb: Agent Starting Run: uzpfw0mo with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 154
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.25
f1 macro,0.2229


0.0016207617531553624

In [17]:
#Don't forget to name the sweep instance   
name = 'rf_ft_liar' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_liar, #CHANGE HERE
          val=embedded_val_liar): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state)
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 12:20:06] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: jb670u0y
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/jb670u0y


[codecarbon WARNING @ 12:20:09] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: ijp0rxys with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 39
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.22897
f1 macro,0.19973


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: pvumfjem with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 190
wandb: 	random_state: 42


...

wandb: Agent Starting Run: qm3lsbkr with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 81
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.22196
f1 macro,0.18738


wandb: Agent Starting Run: owfhn2wa with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 94
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.2352
f1 macro,0.2021


0.001799031014590331